## Tratamento do PPA por regionalização

Nesse notebook são tratados os dados do Plano Plurianual por regionalização (2022 - 2025) da Prefeitura Municipal de São Paulo

In [13]:
import pandas as pd
from utils.load_csv import load_csv
from utils.save_csv import save_csv

In [14]:
df_ppa_reg = load_csv("ppa_reg.csv")

#### Criação "id"

Cria variável "id" concatenando variáveis de código da base de dados para "chave" do merge

In [15]:
#df_ppa_reg["id"] = df_ppa_reg[["codigo orgao",
#                               "codigo unidade",
#                               "codigo programa",
#                               "codigo acao"]].apply(lambda x: "_".join(x.astype(str)), axis = 1)

In [16]:
#cols = ["id"] + [col for col in df_ppa_reg.columns if col != "id"]

#df_ppa_reg = df_ppa_reg[cols]

#### Transformação das variáveis de planejamento orçamentário

Transforma as variáveis de planejamento orçamentário da base de dados de string para float

In [17]:
cols_plan_recurso = ["valor 2022",
                     "valor 2023",
                     "valor 2024",
                     "valor 2025"]

In [18]:
for col in cols_plan_recurso:
    df_ppa_reg[col] = (df_ppa_reg[col]
                         .str.replace(".", "")
                         .str.replace(",", ".")
                         .astype(float))

#### Filtro regionalizável

Filtra a base de dados para conter apenas despesas regionalizáveis

In [19]:
supra_subprefs = ["Supra Subprefeitura",
                  "Supra Subprefeitura Leste",
                  "Supra Subprefeitura Oeste",
                  "Supra Subprefeitura Norte",
                  "Supra Subprefeitura Sul",
                  "Supra Subprefeitura Centro"]

In [20]:
df_ppa_reg = df_ppa_reg[~df_ppa_reg["descricao prefeitura regional"].isin(supra_subprefs)]

#### Normalização das subprefeituras

Trata os nomes das subprefreituras na base de dados para estar igual aos nomes do shapefile das subprefeituras

In [21]:
mapper = {"Subprefeitura Sé": "SE",
          "Subprefeitura Vila Mariana": "VILA MARIANA",
          "Subprefeitura Santana/Tucuruvi": "SANTANA-TUCURUVI",
          "Subprefeitura Campo Limpo": "CAMPO LIMPO",
          "Subprefeitura Itaim Paulista": "ITAIM PAULISTA",
          "Subprefeitura Pirituba/Jaraguá": "PIRITUBA-JARAGUA",
          "Subprefeitura Sapopemba": "SAPOPEMBA",
          "Subprefeitura Capela do Socorro": "CAPELA DO SOCORRO",
          "Subprefeitura Lapa": "LAPA",
          "Subprefeitura Jaçanã/Tremembé": "JACANA-TREMEMBE",
          "Subprefeitura Penha": "PENHA",
          "Subprefeitura São Miguel Paulista": "SAO MIGUEL",
          "Subprefeitura Perus/Anhanguera": "PERUS",
          "Subprefeitura Butantã": "BUTANTA",
          "Subprefeitura de Guaianases": "GUAIANASES",
          "Subprefeitura São Mateus": "SAO MATEUS",
          "Subprefeitura Cidade Ademar": "CIDADE ADEMAR",
          "Subprefeitura de Vila Prudente": "VILA PRUDENTE",
          "Subprefeitura Ipiranga": "IPIRANGA",
          "Subprefeitura Pinheiros": "PINHEIROS",
          "Subprefeitura Casa Verde/Cachoeirinha": "CASA VERDE-CACHOEIRINHA",
          "Subprefeitura Freguesia/Brasilândia": "FREGUESIA-BRASILANDIA",
          "Subprefeitura M'Boi Mirim": "M BOI MIRIM",
          "Subprefeitura Vila Maria/Vila Guilherme": "VILA MARIA-VILA GUILHERME",
          "Subprefeitura Aricanduva/Formosa/Carrão": "ARICANDUVA-FORMOSA-CARRAO",
          "Subprefeitura Santo Amaro": "SANTO AMARO",
          "Subprefeitura Mooca": "MOOCA",
          "Subprefeitura Ermelino Matarazzo": "ERMELINO MATARAZZO",
          "Subprefeitura Cidade Tiradentes": "CIDADE TIRADENTES",
          "Subprefeitura Itaquera": "ITAQUERA",
          "Subprefeitura Jabaquara": "JABAQUARA",
          "Subprefeitura Parelheiros": "PARELHEIROS"}

In [22]:
df_ppa_reg.loc[:, "descricao prefeitura regional"] = df_ppa_reg["descricao prefeitura regional"].map(mapper)

#### Salvar

In [23]:
pd.set_option("display.max_columns", None)

df_ppa_reg.sample(10)

,codigo orgao,descricao orgao,codigo unidade,descricao unidade,codigo programa,descricao programa,codigo acao,descricao acao,codigo da,descricao da,codigo meta,descricao meta,codigo regiao,descricao regiao,codigo prefeitura regional,descricao prefeitura regional,codigo distrito,descricao distrito,codigo sequencial da,codigo produto,descricao produto,descricao unidade medida,quantidade 2022,valor 2022,quantidade 2023,valor 2023,quantidade 2024,valor 2024,quantidade 2025,valor 2025
5749,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,6702,Políticas de Audiovisual,777.04.20.00.001,Subprefeitura Parelheiros,777,Despesa Regionalizável,4,Sul,20,PARELHEIROS,0,Supra-Distrital,1,2904,Atividades culturais realizadas,ações,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
6294,29,Secretaria Municipal de Urbanismo e Licenciamento,20,Operação Urbana Consorciada Água Branca,3002,Acesso à Moradia Adequada,3357,Urbanização de Favelas,777.05.09.00.001,Subprefeitura Sé,777,Despesa Regionalizável,5,Centro,9,SE,0,Supra-Distrital,1,2853,Famílias beneficiadas com obras de urbanização...,famíl,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
1657,16,Secretaria Municipal de Educação,10,Gabinete do Secretário,3012,"Participação, Transparência e Controle Social ...",2803,Manutenção e Operação dos Conselhos e Espaços ...,777.01.01.00.001,Subprefeitura Perus/Anhanguera,777,Despesa Regionalizável,1,Norte,1,PERUS,0,Supra-Distrital,1,2642,Conselhos e espaços participativos mantidos,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
11442,99,Fundo Municipal de Iluminação Pública,10,Fundo Municipal de Iluminação Pública,3022,Requalificação e Promoção da Ocupação dos Espa...,6027,Contraprestação de Parceria Público-Privada (P...,777.04.12.00.001,Subprefeitura Vila Mariana,777,Despesa Regionalizável,4,Sul,12,VILA MARIANA,0,Supra-Distrital,1,2977,Operação/Serviço mantido,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
6716,34,Secretaria Municipal de Direitos Humanos e Cid...,10,Gabinete do Secretário,3007,Promoção dos Direitos da População Idosa,4330,Manutenção e Operação dos Equipamentos Público...,777.04.12.00.001,Subprefeitura Vila Mariana,777,Despesa Regionalizável,4,Sul,12,VILA MARIANA,0,Supra-Distrital,1,2883,Pessoas idosas atendidas pelo Polo Cultural do...,pesso,"1,00",34906.0,"1,00",37431.0,1,38709.0,1,41046.0
10689,98,Fundo de Desenvolvimento Urbano,22,FUNDURB - Secretaria Municipal de Infraestrutu...,3008,Gestão dos Riscos e Promoção da Resiliência a ...,5013,Intervenções no Sistema de Drenagem,777.04.12.00.001,Subprefeitura Vila Mariana,777,Despesa Regionalizável,4,Sul,12,VILA MARIANA,0,Supra-Distrital,1,2881,Áreas inundáveis do município de São Paulo (km²),quilô,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
5208,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,6388,Rádios Comunitárias - Lei nº 16.572/2016,777.02.24.00.001,Subprefeitura Itaim Paulista,777,Despesa Regionalizável,2,Leste,24,ITAIM PAULISTA,0,Supra-Distrital,1,2898,Proponentes com projetos fomentados e/ou incen...,propo,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
6486,30,Secretaria Municipal de Desenvolvimento Econôm...,10,Gabinete do Secretário,3016,Nutrição e Promoção da Segurança Alimentar,4470,Manutenção e Operação de Centro de Referência ...,777.03.11.00.001,Subprefeitura Pinheiros,777,Despesa Regionalizável,3,Oeste,11,PINHEIROS,0,Supra-Distrital,1,2977,Operação/Serviço mantido,unida,"0,00",107700.0,"0,00",136669.0,0,137635.0,0,140243.0
2911,20,Secretaria Municipal de Mobilidade e Trânsito,10,Gabinete do Secretário,3009,Melhoria da Mobilidade Urbana Universal,3757,Implantação de Projetos de Redesenho Urbano pa...,777.04.19.00.001,Subprefeitura Capela do Socorro,777,Despesa Regionalizável,4,Sul,19,CAPELA DO SOCORRO,0,Supra-Distrital,1,2737,Número de novos projetos de redesenho urbano p...,unida,"2,00",81.0,"2,00",1814808.0,2,1923231.0,2,2075089.0
8774,87,Fundo Municipal de Desenvolvimento de Trânsito,10,Fundo Municipal de Desenvolvimento de Trânsito,3009,Melhoria da Mobilidade Urbana Un

In [24]:
save_csv(df_ppa_reg, "ppa_reg.csv")

Base salva em data\ppa_reg.csv
